In [1]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [2]:
class DeepQNetwork(nn.Module):
  def __init__(self, lr, input_dims, fc1_dims, fc2_dims, n_actions):
    super(DeepQNetwork, self).__init__()
    self.input_dims=input_dims
    self.fc1_dims=fc1_dims
    self.fc2_dims=fc2_dims
    self.n_actions=n_actions
    self.fc1=nn.Linear(*self.input_dims, self.fc1_dims)
    #nn.init.xavier_uniform_(self.fc1.weight)
    self.fc2=nn.Linear(self.fc1_dims, self.fc2_dims)
    #nn.init.xavier_uniform_(self.fc2.weight)
    self.fc3=nn.Linear(self.fc2_dims, self.n_actions)
    #nn.init.xavier_uniform_(self.fc3.weight)
    self.optimizer=optim.Adam(self.parameters(), lr=lr)
    self.loss=nn.MSELoss()
    self.device=T.device('cuda:0' if T.cuda.is_available() else 'cpu')
    print(self.device)
    self.to(self.device)

  def forward(self, state):
    x=F.relu(self.fc1(state))
    x=F.relu(self.fc2(x))
    actions=self.fc3(x)

    return actions
#simple network sigmoid - because RelU gives an error near zero

In [3]:
# Python Script that returns axis coordinates of a Duet RepRap based  printer
#two functions get coords and move 

import requests
import json
import sys

endpoint2='/rr_status?type=1'   # RRF2 request
endpoint3='/machine/status'     # RRF3 request
endpointA=endpoint2             # Active
command_coord_move='/rr_gcode?gcode=g1 '
status_M='I'
X=0.0
Y=0.0
Z=0.0
ip_adress='http://192.168.0.1'
def getCoords(base_url):
    global endpointA
    if (endpointA == endpoint2):
        try:
            r = requests.get(f'{base_url}{endpointA}')
            j=json.loads(r.text)
            jc=j['coords']['xyz']
            ret=json.loads('{}')
            for i in range(0,len(jc)):
                ret[ 'xyz'[i] ] = jc[i]
            status_M=j['status']
            #print(status_M)
            return(ret, status_M)
        except:
            endpointA = endpoint3
    if (endpointA == endpoint3):    
        try:
            r = requests.get(f'{base_url}{endpointA}')
            j=json.loads(r.text)
            ja=j['result']['move']['axes']
            jd=j['result']['move']['drives']
            ad=json.loads('{}')
            for i in range(0,len(ja)):
                ad[ ja[i]['letter'] ] = ja[i]['drives'][0]
            ret=json.loads('{}')
            for i in range(0,len(ja)):
                ret[ ja[i]['letter'] ] = jd[i]['position']
            return(ret)
        except:
            print(base_url," does not appear to be a RRF2 or RRF3 printer", file=sys.stderr)
            raise
def move(base_url, coord_axes,coordinate,velocity):
    try:
        getCoords(base_url)
        i=0
        if (coord_axes=='x' or 'y' or 'z' and status_M=='I' ):
           r = requests.get(f'{base_url}{command_coord_move}y{coordinate} f{velocity}') 
           j=json.loads(r.text)
           #print('j')
           #print(f'{base_url}{command_coord_move}x{coordinate} f{velocity}')
           #jc=j['coords']['xyz']
           #ret=json.loads('{}')
        while(getCoords(base_url)[1]!='I'):
            #print("Do!")
            i=i+1
    except:
        print("error")

# Test cases
#print(getCoords('http://192.168.7.100'))            
#print(getCoords('http://127.0.0.1'))            
print(getCoords('http://192.168.0.1')) 

({'x': 0.0, 'y': 11.0, 'z': 0.0}, 'I')


In [4]:
def init_UDP():
    import socket
    UDP_IP = "127.0.0.1"
    UDP_PORT = 61557
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
    sock.bind((UDP_IP, UDP_PORT))
    return sock
def read_UDP_I(sock):
    data, addr = sock.recvfrom(1024) # buffer size is 1024 bytes
    my_str=data.decode('utf-8')
    result=my_str.split()
    #print(result[0])
    return int(result[1])

In [5]:
sock_m=init_UDP()

In [6]:
import torch
import torchvision
torch.cuda.is_available()

True

In [7]:
def reward_f(x, x0, Period, t, A , B):
    #A - oscillations amplitude, global usually, B linear trend amplitude, global
    #return T.tensor(np.exp(-1*(x - x0)**2))*100
    #return T.tensor(((1/(((x - x0)**2)+0.01))-90)/100)
    #return T.tensor((-1*((x +t/B- x0*((1-A)+A*np.sin(t/Period)))**2))/1)
    return T.tensor(read_UDP_I(sock_m)/1000000)

In [8]:
B=T.tensor([-2.0,3.0], dtype=T.float32 )

A=T.tensor([-1.0,4.0], dtype=T.float32 )

print(B[0],A[0])

B[0]=A[0]
print(B[0].item())

tensor(-2.) tensor(-1.)
-1.0


In [9]:

# exp gives low difference in reward near boundaries
# check all - nothing works
#time_global - global, for simulation, Amplitude - global amplitude for simulation, GlobalPeriod, x0=GlobalX
class Enviroment():
  def __init__(self, d_z=0.2, Imax=-100.0):
    self.d_z=d_z
    self.max_val=10
    self.Imax=Imax
    self.reward=0
    self.state_space = np.arange(0, self.max_val, self.d_z) #  from 0 to 10 in tests
    self.state=T.tensor([1.0,-100.0], dtype=T.float32)

  def reset(self, x_new, I, Imax):
    self.reward=0
    self.state[0]=x_new
    self.Imax=Imax
    self.state[1]=I

    return T.tensor([x_new, I], dtype=T.float32)
    # init 

    # 1=> move to 10 -1 => move to 0; 0 => wait
  def step(self, action):
    previous=T.clone(self.state)
    next_state=T.clone(self.state)
    #print(self.Imax)
    if action == 1:
        if self.state[0].item()+self.d_z> self.max_val:
            next_state[0] = self.state[0]
            next_state[1]=reward_f(next_state[0], GlobalX, GlobalPeriod, GlobalTime, Amplitude, BAmp)
            self.reward=T.tensor(-2)
            return(next_state, self.reward, False, False)
        else:
            next_state[0]=self.state[0]+self.d_z
            next_state[1]=reward_f(next_state[0], GlobalX, GlobalPeriod, GlobalTime, Amplitude, BAmp)
        #print(next_state[0], previous[0])  
    elif action == -1:
        if self.state[0].item()-self.d_z < 0:
            next_state[0] = self.state[0]
            next_state[1]=reward_f(next_state[0], GlobalX, GlobalPeriod, GlobalTime, Amplitude, BAmp)
            self.reward=T.tensor(-2)
            return(next_state, self.reward, False, False)
        else:
            next_state[0]=self.state[0]-self.d_z
            next_state[1]=reward_f(next_state[0], GlobalX, GlobalPeriod, GlobalTime, Amplitude, BAmp)
    else:
        next_state[0] = self.state[0]
        next_state[1]=reward_f(next_state[0], GlobalX, GlobalPeriod, GlobalTime, Amplitude, BAmp)
        if (abs(self.Imax-next_state[1].item())<0.01):
            self.reward=T.tensor(0.05)
        else:
            self.reward=T.tensor(-1)
        #if (next_state[1]>self.Imax):
            #self.Imax=next_state[1]
        return(next_state, self.reward, False, False)
    
    if (next_state[1].item()>previous[1].item()):
        self.reward=T.tensor(1)
        #print("Here", self.reward)
    else:
        self.reward=T.tensor(-1)
    self.state=T.clone(next_state) 
    #print(next_state[1]>previous[1], self.reward)
    #if (next_state[1]>self.Imax):
        
        #self.Imax=next_state[1]   
    #print(next_state[1],self.Imax)    
    return (next_state, self.reward*abs(self.Imax-next_state[1].item()*5), False, False)
  

  


In [10]:
class Agent():
  def __init__(self, gamma, epsilon, lr, input_dims, batch_size,  max_mem_size=100000, eps_end=0.01, eps_dec=5e-4):
    # works better for 0.5; epsilon decays from max value to eps end with eps_decay rate
    # it store max_mem_size items 
    self.gamma=gamma
    self.epsilon=epsilon
    self.eps_min=eps_end
    self.eps_dec=eps_dec
    self.lr=lr
    self.action_space=[0, 1, -1]
    #self.action_space=[1, -1]
    #self.action_space=[1, -1, 0] # forward, backward, wait
    self.mem_size=batch_size
    self.batch_size=batch_size
    self.mem_cntr=0
    self.n_actions=len(self.action_space)
    #print(self.n_actions)
    self.Q_eval=DeepQNetwork(self.lr, n_actions=self.n_actions, input_dims=input_dims, fc1_dims=256, fc2_dims=256) # change in dims does not change behaviour
    self.state_memory=np.zeros((self.mem_size, *input_dims), dtype=np.float32) # init memory
    self.new_state_memory=np.zeros((self.mem_size, *input_dims), dtype=np.float32) 
    self.action_memory=np.zeros(self.mem_size, dtype=np.float32)
    self.reward_memory=np.zeros(self.mem_size, dtype=np.float32)
    self.terminal_memory=np.zeros(self.mem_size, dtype=bool)

  def store_transition(self, state, action, reward, state_, done):
    index=self.mem_cntr % self.mem_size
    self.state_memory[index]=state
    self.new_state_memory[index]=state_
    self.reward_memory[index]=reward
    self.action_memory[index]=action
    self.terminal_memory[index]=done

    self.mem_cntr += 1

  def choose_action(self, observation):
    if np.random.random() > self.epsilon:
      #state=T.tensor([observation]).to(self.Q_eval.device)

      state=observation.to(self.Q_eval.device)
      actions=self.Q_eval.forward(state)
      action=self.action_space[T.argmax(actions).item()]
      #print("Random!")
      #print(state, actions, action, T.argmax(actions).item())
    else:
      action=np.random.choice(self.action_space)
      #print("Randpm ",action)
    return action

  def learn(self):
    if self.mem_cntr < self.batch_size:
      return
    
    self.Q_eval.optimizer.zero_grad()

    max_mem= min(self.mem_cntr, self.mem_size)
    batch= np.random.choice(max_mem, self.batch_size, replace=False)

    batch_index = np.arange(self.batch_size, dtype=np.int32)
    state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
    new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
    reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
    terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)

    action_batch=self.action_memory[batch]
    #print(state_batch, batch_index, action_batch)
    q_eval=self.Q_eval.forward(state_batch)[batch_index, action_batch]
    #print(self.Q_eval.forward(state_batch))
    q_next=self.Q_eval.forward(new_state_batch)
    q_next[terminal_batch]=0.0

    q_target=reward_batch+self.gamma*T.max(q_next, dim=1)[0]
    #print(q_target, q_next)
    loss=self.Q_eval.loss(q_target,q_eval).to(self.Q_eval.device)
    #print(q_target, q_next, loss.item())
    #print(loss.item())
    #print(q_target)
    loss.backward()
    self.Q_eval.optimizer.step()

    self.epsilon=self.epsilon-self.eps_dec if self.epsilon > self.eps_min \
                  else self.eps_min 
    return loss

In [ ]:
GlobalX=5
GlobalPeriod=100 
GlobalTime=0
Amplitude=0.1
BAmp=100000

print("Start!")
env=Enviroment(0.01, 0.0)
agent=Agent(gamma=0.99, epsilon=1.0, lr=1e-3, input_dims=[2], batch_size=1024, eps_end=0.01, eps_dec=5e-3)
#action=agent.choose_action(observation)
#env.step(action)
#env.step(action)
#observation=env.reset(5.0)
#print(reward)
observation=env.reset(3.0, 0.0, 0.6)
#self, x_new, x0,t,A,T
i=0
current_score=100
for i in range(150000): 
  action=agent.choose_action(observation)
  #print(observation, action)
  observation_, reward, done, info =env.step(action)
  move('http://192.168.0.1', 'x', observation_[0]*100, 1500)
  #score += reward
  agent.store_transition(observation, action, reward, observation_, done)
  loss2=agent.learn()
  observation=observation_
  GlobalTime+=1
  #print(observation_, reward, action)
  if i%10==1:
    print(i, observation_, action, reward, loss2)
  if (loss2!=None):
    current_score=loss2.item()
  if (current_score<0.0001 and i>10000):
    print("Finish!")
    break
    
 


 

Start!
cuda:0
1 tensor([3.0000, 0.4626]) -1 tensor(1.7130) None
11 tensor([3.0000, 0.4620]) 0 tensor(-1) None
21 tensor([2.9800, 0.4616]) 1 tensor(-1.7082) None
31 tensor([3.0100, 0.4616]) 1 tensor(-1.7082) None
41 tensor([3.0300, 0.4607]) 1 tensor(-1.7034) None
51 tensor([3.0200, 0.4620]) 0 tensor(-1) None
61 tensor([3.0500, 0.4616]) -1 tensor(-1.7082) None
71 tensor([3.0200, 0.4620]) 1 tensor(-1.7098) None
81 tensor([3.0100, 0.4620]) 1 tensor(1.7098) None
91 tensor([2.9800, 0.4616]) -1 tensor(-1.7082) None
101 tensor([2.9800, 0.4616]) 0 tensor(-1) None
111 tensor([2.9900, 0.4616]) -1 tensor(1.7082) None
121 tensor([2.9700, 0.4620]) -1 tensor(-1.7098) None
131 tensor([3.0000, 0.4616]) -1 tensor(-1.7082) None
141 tensor([3.0300, 0.4613]) 1 tensor(1.7066) None
151 tensor([3.0600, 0.4616]) 0 tensor(-1) None
161 tensor([3.0500, 0.4613]) 0 tensor(-1) None
171 tensor([3.0300, 0.4620]) 0 tensor(-1) None
181 tensor([3.0200, 0.4616]) -1 tensor(-1.7082) None
191 tensor([3.0100, 0.4613]) 0 tenso

1321 tensor([4.9800, 0.5324]) 1 tensor(2.0622) tensor(3.1802, device='cuda:0', grad_fn=<MseLossBackward0>)
1331 tensor([5.0800, 0.5324]) 1 tensor(2.0622) tensor(3.1882, device='cuda:0', grad_fn=<MseLossBackward0>)
1341 tensor([5.1800, 0.5321]) 1 tensor(2.0606) tensor(3.2011, device='cuda:0', grad_fn=<MseLossBackward0>)
1351 tensor([5.2800, 0.5321]) 1 tensor(-2.0606) tensor(3.2173, device='cuda:0', grad_fn=<MseLossBackward0>)
1361 tensor([5.3800, 0.5315]) 1 tensor(-2.0574) tensor(3.2288, device='cuda:0', grad_fn=<MseLossBackward0>)
1371 tensor([5.4800, 0.5324]) 1 tensor(2.0622) tensor(3.2354, device='cuda:0', grad_fn=<MseLossBackward0>)
1381 tensor([5.5700, 0.5321]) 1 tensor(-2.0606) tensor(3.2530, device='cuda:0', grad_fn=<MseLossBackward0>)
1391 tensor([5.6700, 0.5321]) 1 tensor(2.0606) tensor(3.2587, device='cuda:0', grad_fn=<MseLossBackward0>)
1401 tensor([5.7700, 0.5315]) 1 tensor(2.0574) tensor(3.2602, device='cuda:0', grad_fn=<MseLossBackward0>)
1411 tensor([5.8700, 0.5308]) 1 te

2091 tensor([8.8701, 0.5347]) -1 tensor(2.0734) tensor(3.4568, device='cuda:0', grad_fn=<MseLossBackward0>)
2101 tensor([8.7701, 0.5331]) -1 tensor(-2.0654) tensor(3.4527, device='cuda:0', grad_fn=<MseLossBackward0>)
2111 tensor([8.6701, 0.5321]) -1 tensor(-2.0606) tensor(3.4524, device='cuda:0', grad_fn=<MseLossBackward0>)
2121 tensor([8.5701, 0.5324]) -1 tensor(-2.0622) tensor(3.4697, device='cuda:0', grad_fn=<MseLossBackward0>)
2131 tensor([8.4701, 0.5331]) -1 tensor(2.0654) tensor(3.4782, device='cuda:0', grad_fn=<MseLossBackward0>)
2141 tensor([8.4101, 0.5334]) 1 tensor(2.0670) tensor(3.4881, device='cuda:0', grad_fn=<MseLossBackward0>)
2151 tensor([8.4301, 0.5334]) 1 tensor(-2.0670) tensor(3.4795, device='cuda:0', grad_fn=<MseLossBackward0>)
2161 tensor([8.4901, 0.5340]) -1 tensor(2.0702) tensor(3.4888, device='cuda:0', grad_fn=<MseLossBackward0>)
2171 tensor([8.5501, 0.5356]) 1 tensor(2.0782) tensor(3.4920, device='cuda:0', grad_fn=<MseLossBackward0>)
2181 tensor([8.5901, 0.5347

2851 tensor([8.3401, 0.5350]) -1 tensor(2.0750) tensor(3.3237, device='cuda:0', grad_fn=<MseLossBackward0>)
2861 tensor([8.3601, 0.5347]) 1 tensor(-2.0734) tensor(3.3519, device='cuda:0', grad_fn=<MseLossBackward0>)
2871 tensor([8.3601, 0.5350]) 1 tensor(-2.0750) tensor(3.3960, device='cuda:0', grad_fn=<MseLossBackward0>)
2881 tensor([8.3601, 0.5360]) -1 tensor(-2.0798) tensor(3.4375, device='cuda:0', grad_fn=<MseLossBackward0>)
2891 tensor([8.3801, 0.5347]) 1 tensor(-2.0734) tensor(3.4765, device='cuda:0', grad_fn=<MseLossBackward0>)
2901 tensor([8.3601, 0.5353]) 1 tensor(2.0766) tensor(3.5182, device='cuda:0', grad_fn=<MseLossBackward0>)
2911 tensor([8.3801, 0.5350]) -1 tensor(2.0750) tensor(3.5566, device='cuda:0', grad_fn=<MseLossBackward0>)
2921 tensor([8.3601, 0.5347]) -1 tensor(-2.0734) tensor(3.6058, device='cuda:0', grad_fn=<MseLossBackward0>)
2931 tensor([8.3801, 0.5356]) -1 tensor(2.0782) tensor(3.6436, device='cuda:0', grad_fn=<MseLossBackward0>)
2941 tensor([8.3801, 0.5353

3611 tensor([8.2101, 0.5353]) -1 tensor(-2.0766) tensor(3.9593, device='cuda:0', grad_fn=<MseLossBackward0>)
3621 tensor([8.2101, 0.5363]) 1 tensor(2.0814) tensor(3.9588, device='cuda:0', grad_fn=<MseLossBackward0>)
3631 tensor([8.2101, 0.5360]) 1 tensor(2.0798) tensor(3.9471, device='cuda:0', grad_fn=<MseLossBackward0>)
3641 tensor([8.1901, 0.5356]) -1 tensor(2.0782) tensor(3.9546, device='cuda:0', grad_fn=<MseLossBackward0>)
3651 tensor([8.1901, 0.5360]) -1 tensor(2.0798) tensor(3.9515, device='cuda:0', grad_fn=<MseLossBackward0>)
3661 tensor([8.2101, 0.5360]) 1 tensor(-2.0798) tensor(3.9793, device='cuda:0', grad_fn=<MseLossBackward0>)
3671 tensor([8.1901, 0.5347]) -1 tensor(-2.0734) tensor(3.9489, device='cuda:0', grad_fn=<MseLossBackward0>)
3681 tensor([8.2101, 0.5360]) 1 tensor(-2.0798) tensor(3.9453, device='cuda:0', grad_fn=<MseLossBackward0>)
3691 tensor([8.2101, 0.5360]) -1 tensor(2.0798) tensor(3.9481, device='cuda:0', grad_fn=<MseLossBackward0>)
3701 tensor([8.1901, 0.5350]

...

http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer


error
error
16951 tensor([8.3901, 0.4216]) 1 tensor(-1.5079) tensor(1.2918, device='cuda:0', grad_fn=<MseLossBackward0>)
error
error
error
error


http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer


error
error
error
error
error
error
16961 tensor([8.4901, 0.4203]) 1 tensor(-1.5015) tensor(1.2735, device='cuda:0', grad_fn=<MseLossBackward0>)
error
error


http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer
http://192.168.0.1  does not appear to be a RRF2 or RRF3 printer


In [ ]:
GlobalX=5
GlobalPeriod=100
GlobalTime=0
Amplitude=0.1
BAmp=5000
import matplotlib.pyplot as plt


env=Enviroment(0.01, 0.0)
#agent=Agent(gamma=0.5, epsilon=1.0, lr=1e-4, input_dims=[2], batch_size=1024, eps_end=0.01, eps_dec=5e-4)
#action=agent.choose_action(observation)
#env.step(action)
#env.step(action)
#observation=env.reset(5.0)
#print(reward)
observation=env.reset(0.0, 0.0, 0.6)
resulat_array=[]
#self, x_new, x0,t,A,T
for i in range(10000):
  action=agent.choose_action(observation)
  #print(observation, action)
  observation_, reward, done, info =env.step(action)
  move('http://192.168.0.1', 'x', observation_[0]*100, 1500)
  #score += reward
  agent.store_transition(observation, action, reward, observation_, done)
  #loss2=agent.learn()
  observation=observation_
  GlobalTime+=1
  #print(observation_, reward, action)
  print(i, observation_[0].item(), observation_[1].item(), action, reward.item())
  resulat_array.append(observation_[1].item())
plt.figure(figsize=(6, 6))
plt.plot(resulat_array)
#plt.plot(rfc_scores, label='RandomForest')
#plt.plot(B2)
plt.title('Intensity',size=14)
plt.xlabel('Step', size=14)
plt.ylabel('accuracy', size=14)
plt.legend()
#plt.xticks(np.arange(1, x_train.shape[1]))
plt.show()

In [ ]:
при параметрах
env=Enviroment(0.1, 0.0)
agent=Agent(gamma=0.99, epsilon=1.0, lr=1e-4, input_dims=[2], batch_size=1024, eps_end=0.01, eps_dec=5e-4)

награда =2 даблюлается лмелшелие осоиоояоий 2x